In [1]:
from cryocooler_avc_api import *
import serial.tools.list_ports
from datetime import datetime,timedelta
from IPython.display import clear_output
#number handling
import pandas as pd
import numpy as np
#encoding decodning
from tqdm.auto import tqdm
#file handling and OS

import sys

#serial connection 
import serial
from serial import SerialException
import serial.tools.list_ports

# time and datetime package
import time
from time import sleep


#Lakehore package 
from lakeshore import Model350


import sys

In [2]:
def signal_handler(signal, frame):
    print("\nprogram exiting gracefully")
    sys.exit(0)


def lakeshore_initialize():
    try: 
        myinstrument =Model350()
        print("connection to Lakeshore successful")
        return(myinstrument)
    except: 
        print("Lakeshore controller not found")
        

def get_temp_data():
    import datetime
    data = {
        'tempA_log': [],
        'tempB_log': [],
        'tempC_log': [],
        
        'tempD_log': [],
        'time':[]
        }
        
    # data['tempA_log'].append(np.float32(myinstrument.query('KRDG? A')))
    time.sleep(0.1)
    data['tempA_log'].append(np.float32(myinstrument.query('KRDG? A')))
    time.sleep(0.1)
    data['tempB_log'].append(np.float32(myinstrument.query('KRDG? B')))
    time.sleep(0.1)
    data['tempC_log'].append(np.float32(myinstrument.query('KRDG? C')))
    time.sleep(0.1)
    data['tempD_log'].append(np.float32(myinstrument.query('KRDG? D')))
    data['time'].append(datetime.datetime.now())
    return data
def log_temp_data_step_duration(step,duration):
    import datetime
    start_time=datetime.datetime.now()
    log_duration = float(duration) #seconds
    log_step=float(step) #seconds 
    df = pd.DataFrame(get_temp_data())
    while datetime.datetime.now()-start_time<datetime.timedelta(seconds=log_duration):
        # print("its true")
        df2=pd.DataFrame(get_temp_data())
        df=df.append(df2,ignore_index = True)
        sleep(log_step)
    return df
def log_temp_data_step_duration(step,duration):
    import datetime
    start_time=datetime.datetime.now()
    log_duration = float(duration) #seconds
    log_step=float(step) #seconds 
    df = pd.DataFrame(get_temp_data())
    while datetime.datetime.now()-start_time<datetime.timedelta(seconds=log_duration):
        # print("its true")
        df2=pd.DataFrame(get_temp_data())
        df=df.append(df2,ignore_index = True)
        sleep(log_step)
    return df
def warmup_detector(temperature=290,heaterrange=5): 
    """Warm up with heater to 290 K"""
    myinstrument=lakeshore_initialize()
    myinstrument.command(f'SETP 1,{temperature}')
    time.sleep(1)
    myinstrument.device_serial.close()
    myinstrument.device_serial.open()
    myinstrument.command(f'RANGE 1,{heaterrange}')
    myinstrument.device_serial.close()
    print(f"CF1 target temperature set to {temperature} K. Heater on CF2 started with range = {heaterrange}")

In [3]:
cryo_port:str='COM7'

In [4]:
cooldown_power: float =30 #W

coldtip_temp_steps= [229.007,196.67,164.34,140.0]  # [229.007,196.67,164.34,132.0]
cf1_temp_steps=[255.844,223.51,191.18,178]
det_temp_steps=[280.0,247.7,215.3,183]
cooldown_time=[30,20,20,20] # [25,20,20,20] #
dwell_time=[20,30,30,50] # [10,10,10,20]

In [6]:

""" Nominal Cooling sequence: For cooling the detector based on steps in previous cell"""
start_time=datetime.now()
for idx,step in enumerate(coldtip_temp_steps):
    if idx==0: 
        print(f"Started cryo in Power mode, Power = {cooldown_power}")
        start_time=datetime.now()
        ser0=start_connection(cryo_port)
        set_target_power_output(ser0,30)
        start_cooler_power_control(ser0)
        #set_soft_stop_state(0)
        stop_connection(ser0) 
        print(f"Setting CF1 setpoint to {cf1_temp_steps[idx]}")
        myinstrument=lakeshore_initialize()
        myinstrument.command(f'SETP 1,{cf1_temp_steps[idx]}')
        time.sleep(1)
        print("Setting CF2 heater range to Range5")
        myinstrument.device_serial.close()
        myinstrument.device_serial.open()
        myinstrument.command(f'RANGE 1,5')
        myinstrument.device_serial.close() 
        print(f"Cooling down for {cooldown_time[idx]} minutes")
        time.sleep(cooldown_time[idx]*60)
        #stop instead of setpoint temperature. 
        print(f"Stopping Cryocooler for {dwell_time[idx]}minutes")
        ser0=start_connection(cryo_port)
        stop_cooler(ser0)
        stop_connection(ser0) 
        time.sleep((dwell_time[idx])*60)
    elif idx==1:
        print(f"Started cryo in Power mode, Power = {cooldown_power}")
        ser0=start_connection(cryo_port)
        set_target_power_output(ser0,30)
        start_cooler_power_control(ser0)
        #set_soft_stop_state(0)
        stop_connection(ser0) 
        print(f"Setting CF1 setpoint to {cf1_temp_steps[idx]}")
        myinstrument=lakeshore_initialize()
        myinstrument.command(f'SETP 1,{cf1_temp_steps[idx]}')
        time.sleep(1)
        print("Setting CF2 heater range to Range5")
        myinstrument.device_serial.close()
        myinstrument.device_serial.open()
        myinstrument.command(f'RANGE 1,5')
        myinstrument.device_serial.close()
        print(f"Cooling down for {cooldown_time[idx]} minutes")
        time.sleep(cooldown_time[idx]*60)
        #stop instead of setpoint temperature. 
        ser0=start_connection(cryo_port)
        stop_cooler(ser0)
        stop_connection(ser0) 
        print(f"Stopping Cryocooler for {dwell_time[idx]}minutes")
        time.sleep((dwell_time[idx])*60)
    elif idx==2: 
        print(f"Started cryo in Power mode, Power = {cooldown_power}")
        ser0=start_connection(cryo_port)
        set_target_power_output(ser0,30)
        start_cooler_power_control(ser0)
        #set_soft_stop_state(0)
        stop_connection(ser0) 
        print(f"Setting CF1 setpoint to {cf1_temp_steps[idx]}")
        myinstrument=lakeshore_initialize()
        myinstrument.command(f'SETP 1,{cf1_temp_steps[idx]}')
        time.sleep(1)
        myinstrument.device_serial.close()
        myinstrument.device_serial.open()
        print("Setting CF2 heater range to Range5")
        myinstrument.command(f'RANGE 1,5')
        myinstrument.device_serial.close()
        print(f"Cooling down for {cooldown_time[idx]} minutes")
        time.sleep(cooldown_time[idx]*60)
        print(f"Switching to Temp control mode with temperature setpoint = {coldtip_temp_steps[idx]} K")
        ser0=start_connection(cryo_port)
        start_cooler_temperature_control(ser0)
        set_target_temperature(ser0,coldtip_temp_steps[idx])
        stop_connection(ser0) 
        print(f"Cryocooler in Temp control mode for {dwell_time[idx]}minutes")
        time.sleep(dwell_time[idx]*60)
    elif idx==3: 
        print(f"Started cryo in Power mode, Power = {cooldown_power}")
        ser0=start_connection(cryo_port)
        set_target_power_output(ser0,30)
        start_cooler_power_control(ser0)
        #set_soft_stop_state(0)
        stop_connection(ser0) 
        print(f"Setting CF1 setpoint to {cf1_temp_steps[idx]}")
        myinstrument=lakeshore_initialize()
        myinstrument.command(f'SETP 1,{cf1_temp_steps[idx]}')
        time.sleep(1)
        print("Setting CF2 heater range to Range5")
        myinstrument.device_serial.close()
        myinstrument.device_serial.open()
        myinstrument.command(f'RANGE 1,5')
        myinstrument.device_serial.close()
        print(f"Cooling down for {cooldown_time[idx]} minutes")
        time.sleep(cooldown_time[idx]*60)
        print(f"Switching to Temp control mode with temperature setpoint = {coldtip_temp_steps[idx]} K")
        ser0=start_connection(cryo_port)
        start_cooler_temperature_control(ser0)
        set_target_temperature(ser0,coldtip_temp_steps[idx])
        stop_connection(ser0) 
        time.sleep(dwell_time[idx]*60)
        print(f"Will have to be shut down with manual command")

Started cryo in Power mode, Power = 30
Cryocooler Controller Connect
Cooler started in Power mode. Set power using PWOUT command
Connection stopped
Setting CF1 setpoint to 255.844
connection to Lakeshore successful
Setting CF2 heater range to Range5
Cooling down for 30 minutes
Stopping Cryocooler for 20minutes
Cryocooler Controller Connect
Cooler Stopped.
Connection stopped
Started cryo in Power mode, Power = 30
Cryocooler Controller Connect
Cooler started in Power mode. Set power using PWOUT command
Connection stopped
Setting CF1 setpoint to 223.51
connection to Lakeshore successful
Setting CF2 heater range to Range5
Cooling down for 20 minutes
Cryocooler Controller Connect
Cooler Stopped.
Connection stopped
Stopping Cryocooler for 30minutes
Started cryo in Power mode, Power = 30
Cryocooler Controller Connect
Cooler started in Power mode. Set power using PWOUT command
Connection stopped
Setting CF1 setpoint to 191.18
connection to Lakeshore successful
Setting CF2 heater range to Range

In [ ]:
# try: 
    
# except Exception: 
#     print(f"Did not compete the cooling cycle. You should restart the cyrocooler. ")

# finally: 
#     print(f"Will have to be shut down with manual command")

Cryocoldtip and Temperatures cf1

140 K, 178 K
135 K, 173 K 


In [5]:
coldtip_temp=140
cf1_temp=168.0 #178, 173, 168, 163, 158,153
print(f"Switching to Temp control mode with temperature setpoint =  {coldtip_temp}K")
ser0=start_connection(cryo_port)
start_cooler_temperature_control(ser0)
set_target_temperature(ser0,coldtip_temp)
stop_connection(ser0) 

print(f"Setting CF1 setpoint to {cf1_temp}")
myinstrument=lakeshore_initialize()
myinstrument.command(f'SETP 1,{cf1_temp}')
time.sleep(1)
print("Setting CF2 heater range to Range5")
myinstrument.device_serial.close()
myinstrument.device_serial.open()
myinstrument.command(f'RANGE 1,5')
myinstrument.device_serial.close()


Switching to Temp control mode with temperature setpoint =  140K
Cryocooler Controller Connect
Cooler started in Temperature mode. Set Temperature using TTARGET command
Connection stopped
Setting CF1 setpoint to 168.0
connection to Lakeshore successful
Setting CF2 heater range to Range5


In [6]:
if input("Are  you sure you want to stop the cryocooler?: Y or  N") == 'Y': 
    ser0=start_connection(cryo_port)
    stop_cooler(ser0)
    stop_connection(ser0)
    temperature=290
    heaterrange=5
    """Warm up with heater to 290 K"""
    myinstrument=lakeshore_initialize()
    myinstrument.command(f'SETP 1,{temperature}')
    time.sleep(1)
    myinstrument.device_serial.close()
    myinstrument.device_serial.open()
    myinstrument.command(f'RANGE 1,{heaterrange}')
    myinstrument.device_serial.close()
    print(f"CF1 target temperature set to {temperature} K. Heater on CF2 started with range = {heaterrange}")

Cryocooler Controller Connect
Cooler Stopped.
Connection stopped
connection to Lakeshore successful
CF1 target temperature set to 290 K. Heater on CF2 started with range = 5
